<a href="https://colab.research.google.com/github/abhi-9971/twitter_sentiment_analysis/blob/main/TWITTER_SENTIMENT_ANALYSIS_USING_ML(MINOR_PROJECT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
#CONTENT OF THE TWITTER DATA

#1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive)
# target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

# ids: The id of the tweet ( 2087)

# date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

# flag: The query (lyx). If there is no query, then this value is NO_QUERY.

# user: the user that tweeted (robotickilldozr)

# text: the text of the tweet (Lyx is cool)

API TO FETCH DATATSET FROM KAGGLE

In [101]:
# # extracting a compressed dataset


# from zipfile import ZipFile
# dataset = 'sentiment140.zip'

# with ZipFile(dataset, 'r') as zip:
#   zip.extractall()
#   print('DATASET EXTRACTED')

IMPORTING THE DEPENDENCIES

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [103]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [104]:
#printing the stopwords in English
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA PROCESSING

In [105]:
#loading the data form csv file to pandas df
twitter_data=pd.read_csv('balanced_data.csv', encoding = 'ISO-8859-1')

In [106]:
#checking the no of rows and columns
a=twitter_data.shape
b=twitter_data.size

print(a)
print(b)


(9999, 6)
59994


In [107]:
twitter_data.head()

,4,1991330743,Mon Jun 01 06:50:34 PDT 2009,NO_QUERY,BRUNETTEBARBIEE,Found the perfect title for my portfolio. its from a song that i love
0,0,1881517921,Fri May 22 04:52:46 PDT 2009,NO_QUERY,petdoctorforum,my new anti-virus ive just installed seems to ...
1,0,2051830441,Fri Jun 05 22:55:06 PDT 2009,NO_QUERY,juliecastle_9,fact: i still love him...you learn from mistak...
2,0,1988997171,Mon Jun 01 00:00:59 PDT 2009,NO_QUERY,sojufied,i think i gotta make a new twitter account cuz...
3,0,2070776908,Sun Jun 07 18:16:45 PDT 2009,NO_QUERY,janicetertel,end of the weekend....back to work tomorrow W...
4,4,2009606301,Tue Jun 02 15:58:40 PDT 2009,NO_QUERY,Kt_Rose517,@mitchelmusso LOVE the album!! reply??


In [108]:
#naming the columns and reading the dataset again

column_names = ['target', 'id' , 'date', 'flag' , 'user' , 'text']
twitter_data=pd.read_csv('balanced_data.csv', names = column_names , encoding = 'ISO-8859-1')

In [109]:
twitter_data.shape



(10000, 6)

In [110]:
twitter_data.head()

,target,id,date,flag,user,text
0,4,1991330743,Mon Jun 01 06:50:34 PDT 2009,NO_QUERY,BRUNETTEBARBIEE,Found the perfect title for my portfolio. its...
1,0,1881517921,Fri May 22 04:52:46 PDT 2009,NO_QUERY,petdoctorforum,my new anti-virus ive just installed seems to ...
2,0,2051830441,Fri Jun 05 22:55:06 PDT 2009,NO_QUERY,juliecastle_9,fact: i still love him...you learn from mistak...
3,0,1988997171,Mon Jun 01 00:00:59 PDT 2009,NO_QUERY,sojufied,i think i gotta make a new twitter account cuz...
4,0,2070776908,Sun Jun 07 18:16:45 PDT 2009,NO_QUERY,janicetertel,end of the weekend....back to work tomorrow W...


In [111]:
#counting the number of missing values
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [112]:
#checking the distribution of our target Column
twitter_data['target'].value_counts()

target
4    5000
0    5000
Name: count, dtype: int64

CONVERT THE TARGET '4' TO '1'

In [113]:
#from google.colab import drive
#drive.mount('/content/drive')

In [114]:
twitter_data.replace({'target':{4:1}} , inplace =True)

In [115]:
#checking the distribution of our target Column
twitter_data['target'].value_counts()

target
1    5000
0    5000
Name: count, dtype: int64

0 --> NEGATIVE TWEET
1 --> POSITIVE TWEET

**STEMMING**


The practice of reducing a word to its fundamental word is called stemming.

Example : actor, actress , acting = act

In [116]:
port_stem = PorterStemmer()


In [117]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ',content )
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in  stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [118]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)  #here we have called our function stemming for the input as CONTENT from the twitter_data

In [119]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,1,1991330743,Mon Jun 01 06:50:34 PDT 2009,NO_QUERY,BRUNETTEBARBIEE,Found the perfect title for my portfolio. its...,found perfect titl portfolio song love
1,0,1881517921,Fri May 22 04:52:46 PDT 2009,NO_QUERY,petdoctorforum,my new anti-virus ive just installed seems to ...,new anti viru ive instal seem effect struggl o...
2,0,2051830441,Fri Jun 05 22:55:06 PDT 2009,NO_QUERY,juliecastle_9,fact: i still love him...you learn from mistak...,fact still love learn mistak hope repeat day
3,0,1988997171,Mon Jun 01 00:00:59 PDT 2009,NO_QUERY,sojufied,i think i gotta make a new twitter account cuz...,think gotta make new twitter account cuz quot ...
4,0,2070776908,Sun Jun 07 18:16:45 PDT 2009,NO_QUERY,janicetertel,end of the weekend....back to work tomorrow W...,end weekend back work tomorrow need longer wee...


In [120]:
print(twitter_data['stemmed_content']) #here we printed the stemmed data

0                  found perfect titl portfolio song love
1       new anti viru ive instal seem effect struggl o...
2            fact still love learn mistak hope repeat day
3       think gotta make new twitter account cuz quot ...
4       end weekend back work tomorrow need longer wee...
                              ...                        
9995                  love onu admiss ba alway good laugh
9996    wait mum friend come gonna teach drive land ro...
9997    lisa veronica hey lisa wait see us look forwar...
9998          got back church broke foot caught holyghost
9999    rafaeloyama hey im hey monday fan song like al...
Name: stemmed_content, Length: 10000, dtype: object


In [121]:
print(twitter_data['target'])

0       1
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    1
9998    0
9999    1
Name: target, Length: 10000, dtype: int64


In [122]:
#Seperate the label and the data
X= twitter_data['stemmed_content'].values
Y= twitter_data['target'].values

In [123]:
print(X)


['found perfect titl portfolio song love'
 'new anti viru ive instal seem effect struggl open anyth internet'
 'fact still love learn mistak hope repeat day' ...
 'lisa veronica hey lisa wait see us look forward pretti reckless well take care'
 'got back church broke foot caught holyghost'
 'rafaeloyama hey im hey monday fan song like album like demi']


In [124]:
print(Y)

[1 0 0 ... 1 0 1]


In [125]:
X_train,X_test,Y_train,Y_test = train_test_split( X , Y , test_size = 0.2 , stratify=Y , random_state=2)

In [126]:
print(X.shape , X_train.shape , X_test.shape)

(10000,) (8000,) (2000,)


In [127]:
print(X_train)

['jbnvfclover haha know'
 'rli wish take nap instead wait peopl come look hous'
 'myinkyfingersau snowyowle honeyjojam snag coffe crumpet snuggl yeah wasp hubbi vanquis nobli'
 ... 'saraacoop nope troop yong bloor workout suppli mean troop'
 'hhume ohh haha ahhh wanna go parti park'
 'ddlovato ye final home hope go perfect']


In [128]:
print(X_test)

['ughhh get work go long day'
 'cant believ come run give quot plenti fish sea quot speech'
 'time shower tidi go go lunch friend lake side hope sun peep littl fun'
 ... 'final finish littl princ soo good'
 'dubarrymcfli yeah would cool want next weekend hurri lol xxx'
 'ok realli want make twatter com quot bitch bite nippl quot']


In [129]:
#conversion from textual to numeric data

vectorizer =  TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [130]:
print(X_train)


  (0, 6264)	0.3793944427283553
  (0, 4711)	0.4372125333387804
  (0, 5711)	0.8154171064665778
  (1, 5140)	0.3126983025176187
  (1, 6753)	0.2462481410649984
  (1, 2278)	0.24264044962221704
  (1, 8623)	0.2745853653955927
  (1, 12320)	0.260936747445089
  (1, 5456)	0.37110509290726273
  (1, 7810)	0.36061186461812417
  (1, 11114)	0.2595422438577077
  (1, 12593)	0.26058461961285856
  (1, 9591)	0.4859996919387574
  (2, 8027)	0.31029122325122677
  (2, 12134)	0.31029122325122677
  (2, 5174)	0.2565592375339515
  (2, 12372)	0.31029122325122677
  (2, 12837)	0.1725045313153249
  (2, 10466)	0.29675454362574705
  (2, 2551)	0.31029122325122677
  (2, 2239)	0.21473504449719114
  (2, 10438)	0.31029122325122677
  (2, 5096)	0.31029122325122677
  (2, 10463)	0.31029122325122677
  (2, 7750)	0.31029122325122677
  :	:
  (7996, 10201)	0.29588269027336456
  (7996, 12377)	0.2540157628478748
  (7997, 1256)	0.32436307150726745
  (7997, 12900)	0.32436307150726745
  (7997, 11766)	0.6487261430145349
  (7997, 9824)	0.324

In [131]:
print(X_test)

  (0, 12663)	0.3066760214890747
  (0, 11947)	0.6762458756425311
  (0, 6746)	0.4397331701720937
  (0, 4464)	0.2777091217470457
  (0, 4364)	0.295222626896164
  (0, 2778)	0.3016576152486661
  (1, 10633)	0.3646733184621627
  (1, 9949)	0.3743863319532119
  (1, 9719)	0.2680756204643496
  (1, 9197)	0.4173791380684621
  (1, 8822)	0.3743863319532119
  (1, 4422)	0.2635842073237334
  (1, 3957)	0.31404358060404725
  (1, 2278)	0.20197968374290903
  (1, 1744)	0.23384168333545424
  (1, 1036)	0.2775527087487682
  (2, 11504)	0.18850383098363913
  (2, 10944)	0.28822921984905864
  (2, 10245)	0.3473690977194854
  (2, 10203)	0.323383388967789
  (2, 8606)	0.3704596787619168
  (2, 6862)	0.2847051654403786
  (2, 6673)	0.25751226281792255
  (2, 6376)	0.3473690977194854
  (2, 5111)	0.20744133701753373
  :	:
  (1996, 3992)	0.24041951390967586
  (1996, 437)	0.2628113158720127
  (1997, 10550)	0.4243011849908711
  (1997, 9015)	0.5956969534864195
  (1997, 6673)	0.36513232142894675
  (1997, 4496)	0.2467343213011989
 

TRAINING THE ML MODEL | LOGISTIC REGRESSION

In [132]:
model = LogisticRegression(max_iter = 1000)


In [133]:
model.fit(X_train , Y_train)

LogisticRegression(max_iter=1000)

MODEL EVALUATION

ACCURACY SCORE

In [134]:
#ACCURACY SCORE ON THE TRAINING DATA
X_train_prediction =  model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train , X_train_prediction)

In [135]:
print('Accuracy Score on the training data : ' , training_data_accuracy)

Accuracy Score on the training data :  0.86975


In [136]:
#ACCURACY SCORE ON THE TEST DATA
X_test_prediction =  model.predict(X_test)
training_data_accuracy= accuracy_score(Y_test , X_test_prediction)

In [137]:
print('Accuracy Score on the test data : ' , training_data_accuracy)

Accuracy Score on the test data :  0.716


SAVING THE TRAINED MODEL

In [138]:
import pickle


In [139]:
filename= 'trained_model.pkl'
pickle.dump(model,open(filename,'wb'))

In [140]:
# vectorizer = TfidfVectorizer(max_features=13004)  # Adjust max_features as needed
# X_train_transformed = vectorizer.fit_transform(X_train)
with open('vectori.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)